## Scrapy: extracting real estate data from emlakjet


While preparing codes in this page, I have utilized following source: https://app.datacamp.com/learn/courses/web-scraping-with-python

In [ ]:
#pip install scrapy

In [ ]:
# Import a scrapy Selector
from scrapy import Selector

# Import requests
import requests

url = 'https://www.emlakjet.com/'

# Create the string html containing the HTML source
html = requests.get(url).content

# Create the Selector object sel from html
sel = Selector(text = html)

# Print out the number of elements in the HTML document
print( "You have found: ", len(sel.xpath('//*')))

In [ ]:
html

In [ ]:
type(html)

In [ ]:
type(sel)

In [ ]:
xpath_for_titles ='//h4'

In [ ]:
sel.xpath(xpath_for_titles).extract()

In [ ]:
len(sel.xpath(xpath_for_titles).extract())

In [ ]:
xpath_for_titles ='//h4/text()'

In [ ]:
titles = sel.xpath(xpath_for_titles).extract()

In [ ]:
titles

In [ ]:
url = 'https://www.emlakjet.com/satilik-konut/istanbul-fatih/?max_fiyat=3250000&min_fiyat=1500000'

# Create the string html containing the HTML source
html = requests.get(url).content

# Create the Selector object sel from html
sel = Selector(text = html)

# Print out the number of elements in the HTML document
print( "You have found: ", len(sel.xpath('//*')))

//*[@id="listing-search-wrapper"]/div[1]/a/div[3]/div/div[1]/h3

//*[@id="listing-search-wrapper"]/div[12]/a/div[3]/div/div[1]/h3

//*[@id="listing-search-wrapper"]/div[24]/a/div[3]/div/div[1]/h3

In [ ]:
xpath_for_titles_of_adverts ='//*[@id="listing-search-wrapper"]/div/a/div[3]/div/div[1]/h3/text()'
titles_of_adverts = sel.xpath(xpath_for_titles_of_adverts).extract()
titles_of_adverts

### Clean your data

In [ ]:
import re

def strip_tags(html):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', html)

# Apply strip_tags to each element in the list
cleaned_strings = [strip_tags(html) for html in titles_of_adverts]

print(cleaned_strings)

### Manipulate searchterms and parameters

In [ ]:
district = input()
maxim = input()
minim = input()

url = 'https://www.emlakjet.com/satilik-konut/istanbul-' + district + '/?max_fiyat=' + maxim + '&min_fiyat=' + minim

html = requests.get(url).content

sel = Selector(text = html)

xpath_for_titles_of_adverts ='//*[@id="listing-search-wrapper"]/div/a/div[3]/div/div[1]/h3'
titles_of_adverts = sel.xpath(xpath_for_titles_of_adverts).extract()

cleaned_strings = [strip_tags(html) for html in titles_of_adverts]

cleaned_strings

In [ ]:
district_list = ["uskudar", "fatih", "umraniye"]
intervals = [0, 5000000, 10000000, 20000000]

output_titles = []

for district in district_list:
    for i in range(len(intervals)-1):
        url = 'https://www.emlakjet.com/satilik-konut/istanbul-' + district + '/?max_fiyat=' + str(intervals[i+1]) + '&min_fiyat=' + str(intervals[i])

        html = requests.get(url).content

        sel = Selector(text = html)

        xpath_for_titles_of_adverts ='//*[@id="listing-search-wrapper"]/div/a/div[3]/div/div[1]/h3/text()'
        titles_of_adverts = sel.xpath(xpath_for_titles_of_adverts).extract()
        for j in titles_of_adverts:
            output_titles.append(j)

In [ ]:
output_titles

In [ ]:
output_titles = [strip_tags(html) for html in output_titles]

output_titles

### Collect links

//*[@id="listing-search-wrapper"]/div[@class='_3qUI9q']/a

In [ ]:
district_list = ["uskudar", "fatih", "umraniye"]
intervals = [0, 5000000, 10000000, 20000000]

output_for_links = []

for district in district_list:
    for i in range(len(intervals)-1):
        url = 'https://www.emlakjet.com/satilik-konut/istanbul-' + district + '/?max_fiyat=' + str(intervals[i+1]) + '&min_fiyat=' + str(intervals[i])

        html = requests.get(url).content

        sel = Selector(text = html)
        xpath_for_links = "//*[@id='listing-search-wrapper']/div[@class='_3qUI9q']/a/@href"
        link_list = sel.xpath(xpath_for_links).extract()
        for j in link_list:
            output_for_links.append(j)

In [ ]:
output_for_links

In [ ]:
first_part_url = "https://www.emlakjet.com/"

In [ ]:
adverts_link_merged = []

for i in output_for_links:
    link = first_part_url + i
    adverts_link_merged.append(link)

In [ ]:
len(adverts_link_merged)

### Collect prices

//*[@id="listing-search-wrapper"]/div[15]/a/div[3]/div/div[3]/div/p/span/span

In [ ]:
district_list = ["uskudar", "fatih", "umraniye"]
intervals = [0, 5000000, 10000000, 20000000]

output_for_prices = []

for district in district_list:
    for i in range(len(intervals)-1):
        url = 'https://www.emlakjet.com/satilik-konut/istanbul-' + district + '/?max_fiyat=' + str(intervals[i+1]) + '&min_fiyat=' + str(intervals[i])

        html = requests.get(url).content

        sel = Selector(text = html)
        xpath_for_prices = "//*[@id='listing-search-wrapper']/div/a/div[3]/div/div[3]/div/p/span/span/text()"
        price_list = sel.xpath(xpath_for_prices).extract()
        for j in price_list:
            output_for_prices.append(j)

In [ ]:
output_for_prices

### Create real_estate dataframe

In [ ]:
dict_real_estate = {'title':output_titles, 'price':output_for_prices, 'link': adverts_link_merged}

In [ ]:
import pandas as pd

In [ ]:
data_real_estate = pd.DataFrame(dict_real_estate)

In [ ]:
data_real_estate

#### Can we go further?

Let's find out building ages for these adverts:


//*[@id="bilgiler"]/div/div[2]/div/div[1]/div[2]/div[5]/div[2]
//*[@id="bilgiler"]/div/div[2]/div/div[1]/div[2]/div[5]/div[2]

In [ ]:
xpath_for_age = "//*[@id='bilgiler']/div/div[2]/div/div[1]/div[2]/div[5]/div[2]/text()"

In [ ]:
age_list = []

for i in range(len(data_real_estate)):
    url = data_real_estate['link'][i]
    html = requests.get(url).content
    sel = Selector(text = html)
    building_age = sel.xpath(xpath_for_age).extract()
    for j in building_age:
        age_list.append(j)

In [ ]:
age_list